<a href="https://colab.research.google.com/github/Chia-Yin-Lee/TextAnalysis/blob/main/text_svm_mnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title show your CPU or GPU details
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4351532307752532726
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14357954560
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11182107385088977769
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [1]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_TA

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_TA


In [3]:
!pip install nlpaug
!pip install transformers

In [2]:
#@title import packages
import pandas as pd
import numpy as np

import datetime as dt
from datetime import datetime
import string
import os
import codecs
import io
from lxml import etree
from dateutil.relativedelta import *
import seaborn as sns; sns.set(style="darkgrid")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re
import pickle
from tqdm.notebook import tqdm
#import nltk
#import transformers
#import nlpaug
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.semi_supervised import LabelPropagation
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

In [3]:
reviews_df = pd.read_csv('clean_reviews_top100.csv')
reviews_df = reviews_df[reviews_df['comment'].notna() & (reviews_df['comment'].str.strip() != '')]
reviews_df.head()

,Unnamed: 0,user,rating,comment,ID,name
0,1,mitnachtKAUBO-I,10.0,hands down my favorite new game of bgg con we ...,30549,Pandemic
1,2,avlawn,10.0,i tend to either love or easily tire of coop g...,30549,Pandemic
2,4,Mease19,10.0,this is an amazing coop game i play mostly wit...,30549,Pandemic
3,5,cfarrell,10.0,hey i can finally rate this game ive been play...,30549,Pandemic
4,8,gregd,10.0,love it great fun with my son plays so far and...,30549,Pandemic


In [4]:
#reviews_df['rating'] = reviews_df['rating'].astype(int)
reviews_df.rename(columns={'Unnamed: 0': 'original_id'}, inplace=True)
reviews_df.head()

<ipython-input-4-cdf0793aeb95>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df.rename(columns={'Unnamed: 0': 'original_id'}, inplace=True)


,original_id,user,rating,comment,ID,name
0,1,mitnachtKAUBO-I,10.0,hands down my favorite new game of bgg con we ...,30549,Pandemic
1,2,avlawn,10.0,i tend to either love or easily tire of coop g...,30549,Pandemic
2,4,Mease19,10.0,this is an amazing coop game i play mostly wit...,30549,Pandemic
3,5,cfarrell,10.0,hey i can finally rate this game ive been play...,30549,Pandemic
4,8,gregd,10.0,love it great fun with my son plays so far and...,30549,Pandemic


In [6]:
def transform_0to1(x):
    if x == 0:
        return 1
    else:
        return x

reviews_df['rating'] = reviews_df['rating'].apply(transform_0to1)

In [5]:
def transform_value(x):
    if x <= 4:
        return -1
    elif x <= 7:
        return 0
    else:
        return 1

reviews_df['rating'] = reviews_df['rating'].apply(transform_value)
reviews_df.head()

,original_id,user,rating,comment,ID,name
0,1,mitnachtKAUBO-I,1,hands down my favorite new game of bgg con we ...,30549,Pandemic
1,2,avlawn,1,i tend to either love or easily tire of coop g...,30549,Pandemic
2,4,Mease19,1,this is an amazing coop game i play mostly wit...,30549,Pandemic
3,5,cfarrell,1,hey i can finally rate this game ive been play...,30549,Pandemic
4,8,gregd,1,love it great fun with my son plays so far and...,30549,Pandemic


In [6]:
#@title split first (8:2)

train_data, test_data = model_selection.train_test_split(reviews_df, test_size = 0.2, random_state = 2023)
print(len(train_data), len(test_data))

403372 100844


In [7]:
train_text = train_data['comment']
test_text = test_data['comment']
train_rating = train_data['rating']
test_rating = test_data['rating']
'''
train_text = train_data['comment'].tolist()
test_text = test_data['comment'].tolist()
train_rating = train_data['rating'].tolist()
test_rating = test_data['rating'].tolist()'''

"\ntrain_text = train_data['comment'].tolist()\ntest_text = test_data['comment'].tolist()\ntrain_rating = train_data['rating'].tolist()\ntest_rating = test_data['rating'].tolist()"

In [8]:
Tfidvectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english')

train_text = Tfidvectorizer.fit_transform(train_text)
test_text = Tfidvectorizer.transform(test_text)

In [29]:
#@title SVC

SVC_model = LinearSVC(random_state = 0)

SVC_model.fit(train_text, train_rating)

predicted_rating_SVC = SVC_model.predict(test_text)
accuracy_SVC = accuracy_score(test_rating, predicted_rating_SVC)
print("Accuracy on the validation set:", accuracy_SVC)
balanced_accuracy_SVC = balanced_accuracy_score(test_rating, predicted_rating_SVC)
print("Balanced Accuracy on the validation set:", balanced_accuracy_SVC)

report_SVC = classification_report(test_rating, predicted_rating_SVC, output_dict = True)
print(report_SVC)

[LibLinear]Accuracy on the validation set: 0.6528201975328229
Balanced Accuracy on the validation set: 0.5024457502394744


In [31]:
with open('SVC.pickle', 'wb') as f:
    pickle.dump(SVC_model, f)

In [ ]:
# Define a grid of hyperparameters to search
param_grid = {
    'C': [0.01, 0.1, 1, 10],        # Regularization parameter
    'kernel': ['linear', 'rbf'],    # Kernel type
}

# Initialize the SVC classifier
svc_classifier = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svc_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_text, train_rating)

# Get the best hyperparameters and the corresponding accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy:.2f}")

In [24]:
#@title Logistic Regression

LR_model = LogisticRegression(max_iter = 100000, solver = 'saga')

LR_model.fit(train_text, train_rating)

predicted_rating_LR = LR_model.predict(test_text)
accuracy_LR = accuracy_score(test_rating, predicted_rating_LR)
print("Accuracy:", accuracy_LR)
balanced_accuracy_LR = balanced_accuracy_score(test_rating, predicted_rating_LR)
print("Balanced Accuracy:", balanced_accuracy_LR)

report_LR = classification_report(test_rating, predicted_rating_LR, output_dict = True)
print(report_LR)

Accuracy: 0.6611102296616557
Balanced Accuracy: 0.5209021774003909


In [9]:
# Define a grid of hyperparameters to search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'penalty': ['l1', 'l2'],  # Regularization type
}

# Initialize the logistic regression classifier
lr_classifier = LogisticRegression(max_iter=10000, random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(lr_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_text, train_rating)

# Get the best hyperparameters and the corresponding accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

Best Hyperparameters: {'C': 1, 'penalty': 'l2'}
Best Accuracy: 0.66


In [25]:
with open('LogicticRegression.pickle', 'wb') as f:
    pickle.dump(LR_model, f)

In [15]:
#@title MultinomialNB

MNB_model = MultinomialNB(alpha = 0.1)

MNB_model.fit(train_text, train_rating)

predicted_rating_MNB = MNB_model.predict(test_text)
accuracy_MNB = accuracy_score(test_rating, predicted_rating_MNB)
print("Accuracy on the validation set:", accuracy_MNB)
balanced_accuracy_MNB = balanced_accuracy_score(test_rating, predicted_rating_MNB)
print("Balanced Accuracy on the validation set:", balanced_accuracy_MNB)

report_MNB = classification_report(test_rating, predicted_rating_MNB, output_dict = True)
report_MNB

Accuracy on the validation set: 0.6392348578001666
Balanced Accuracy on the validation set: 0.4475837767532324


{'-1': {'precision': 0.6117021276595744,
  'recall': 0.03542821934688848,
  'f1-score': 0.0669772859638905,
  'support': 6492},
 '0': {'precision': 0.5858321531925375,
  'recall': 0.4760463312736097,
  'f1-score': 0.5252639569387897,
  'support': 39973},
 '1': {'precision': 0.6649015973876975,
  'recall': 0.831276779639199,
  'f1-score': 0.7388387202222858,
  'support': 54379},
 'accuracy': 0.6392348578001666,
 'macro avg': {'precision': 0.6208119594132698,
  'recall': 0.4475837767532324,
  'f1-score': 0.4436933210416553,
  'support': 100844},
 'weighted avg': {'precision': 0.6301348898960459,
  'recall': 0.6392348578001666,
  'f1-score': 0.6109287955471768,
  'support': 100844}}

In [14]:
# Define a range of alpha values to test
alpha_values = [0.1, 0.5, 1.0, 2.0]
param_grid = {'alpha': alpha_values}

# Initialize the MNB classifier
nb_classifier = MultinomialNB()

# Perform grid search with cross-validation
grid_search = GridSearchCV(nb_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_text, train_rating)

# Get the best alpha value and the corresponding accuracy
best_alpha = grid_search.best_params_['alpha']
best_accuracy = grid_search.best_score_

print(f"Best Alpha: {best_alpha}")
print(f"Best Accuracy: {best_accuracy:.2f}")

Best Alpha: 0.1
Best Accuracy: 0.64


In [16]:
with open('MultinomialNB.pickle', 'wb') as f:
    pickle.dump(MNB_model, f)

## the following is not continued

In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Tokenize and preprocess the text data
def tokenize_text(text):
    return word_tokenize(text.lower())

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reviews_df['comment'], reviews_df['rating'], test_size=0.2, random_state=42)

tagged_data = [TaggedDocument(words=tokenize_text(text), tags=[str(idx)]) for idx, text in enumerate(X_train)]

# Initialize and train the Doc2Vec model
d2v_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
d2v_model.build_vocab(tagged_data)
d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Transform text data into Doc2Vec embeddings
X_train_d2v = [d2v_model.infer_vector(tokenize_text(text)) for text in X_train]

# Train and evaluate the Multinomial Naive Bayes model
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_d2v, y_train)

# Transform the test data into Doc2Vec embeddings
X_test_d2v = [d2v_model.infer_vector(tokenize_text(text)) for text in X_test]

# Predict sentiment on test data
y_pred_nb = nb_classifier.predict(X_test_d2v)

# Evaluate the models
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Accuracy (Multinomial Naive Bayes): {accuracy_nb:.2f}")

print("Classification Report (Multinomial Naive Bayes):\n", classification_report(y_test, y_pred_nb))

ValueError: ignored

In [ ]:
with open('MultinomialNB_D2V.pickle', 'wb') as f:
    pickle.dump(nb_classifier, f)


In [ ]:
# Train and evaluate the Support Vector Classifier (SVC) model
svc_classifier = SVC(kernel='linear', C=1.0)
svc_classifier.fit(X_train_d2v, y_train)

In [ ]:
# Transform the test data into Doc2Vec embeddings
X_test_d2v = [d2v_model.infer_vector(tokenize_text(text)) for text in X_test]

# Predict sentiment on test data
y_pred_svc = svc_classifier.predict(X_test_d2v)

# Evaluate the models
accuracy_svc = accuracy_score(y_test, y_pred_svc)

print(f"Accuracy (Support Vector Classifier): {accuracy_svc:.2f}")
print("Classification Report (Support Vector Classifier):\n", classification_report(y_test, y_pred_svc))

In [ ]:
with open('LinearSVC_D2V.pickle', 'wb') as f:
    pickle.dump(svc_classifier, f)